In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [34]:
class Config():
    input_paths = ['./data_for_training/data/2023-wimbledon-1301-player1.csv']
                #    './data_for_training/data/2023-wimbledon-1301-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1302-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1302-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1303-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1303-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1304-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1304-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1305-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1305-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1306-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1306-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1307-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1307-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1308-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1308-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1309-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1309-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1310-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1310-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1311-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1311-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1312-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1312-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1313-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1313-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1314-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1314-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1315-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1315-player2.csv',
                #    './data_for_training/data/2023-wimbledon-1316-player1.csv',
                #    './data_for_training/data/2023-wimbledon-1316-player2.csv']
    timestep = 10
    batch_size = 32
    feature_size = 16
    hidden_size = 256 # TODO: can change
    output_size = 4
    predict_len = 5
    num_layers = 3 # TODO: can change
    epochs = 10 # TODO: can change
    best_loss = 100
    learning_rate = 0.0003 # TODO: can change
    model_name = 'GRU_for_tennis_momentum_swing'
    save_path = './{}.pth'.format(model_name)

config = Config()

In [35]:

def fetch_data(dataX, dataY, timestep, feature_size, output_size, pred_len):
    myDataX = []
    myDataY = []
    for i in range(0, len(dataX) - timestep):
        target_row = dataY[i + timestep - 1].reshape(-1, output_size)
        res = []
        for j in range(target_row.shape[0]):
            for k in range(target_row.shape[1]):
                if target_row[j][k] == 1:
                    res.append(k)
                    break
        if len(res) != target_row.shape[0]:
            break
        myDataX.append(dataX[i: i + timestep])
        myDataY.append(res)
    myDataX = np.array(myDataX)
    myDataY = np.array(myDataY)

    train_size = int(np.round(0.8 * myDataX.shape[0]))
    trainX = myDataX[:train_size, :].reshape(-1, timestep, feature_size)
    testX = myDataX[train_size:, :].reshape(-1, timestep, feature_size)
    trainY = myDataY[:train_size, :].reshape(-1, pred_len)
    testY = myDataY[train_size:, :].reshape(-1, pred_len)
    return (trainX, trainY, testX, testY)

trainX = []
trainY = []
testX = []
testY = []
dfy = pd.read_csv('./data_for_training/momentum_condition.csv')
dataY = np.array(dfy)
df_idx = pd.read_csv('./data_for_training/match_index.csv')
match_id = np.array(df_idx).reshape(-1)
for i in range(len(match_id)):
    match_id[i] -= 1
pos = 0
match_no = 0

for input_path in config.input_paths:
    dfx = pd.read_csv(input_path)
    dataX = np.array(dfx)
    a, b, c, d = fetch_data(dataX, dataY[pos: match_id[match_no // 2]], config.timestep, config.feature_size, config.output_size, config.predict_len)
    if match_no % 2 == 1:
        pos = match_id[match_no // 2]
    match_no += 1
    for x in a:
        trainX.append(x)
    for y in b:
        trainY.append(y)
    for x in c:
        testX.append(x)
    for y in d:
        testY.append(y)
trainX = np.array(trainX)#.reshape(-1, config.timestep, config.feature_size)
testX = np.array(testX).reshape(-1, config.timestep, config.feature_size)
trainY = np.array(trainY).reshape(-1, config.predict_len)
testY = np.array(testY).reshape(-1, config.predict_len)

x_train_tensor = torch.from_numpy(trainX).to(torch.float32)
y_train_tensor = torch.from_numpy(trainY).to(torch.long)

x_test_tensor = torch.from_numpy(testX).to(torch.float32)
y_test_tensor = torch.from_numpy(testY).to(torch.long)

train_data = TensorDataset(x_train_tensor, y_train_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, config.batch_size, False)
test_loader = DataLoader(test_data, config.batch_size, False)


In [36]:
# GRU
class GRURNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size, output_size)
        self.fc2 = nn.Linear(self.hidden_size, output_size)
        self.fc3 = nn.Linear(self.hidden_size, output_size)
        self.fc4 = nn.Linear(self.hidden_size, output_size)
        self.fc5 = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, input_seq):
        batch_size = input_seq.shape[0]
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        output, _ = self.gru(input_seq,h_0)
        pred1 = self.fc1(output)
        pred2 = self.fc2(output)
        pred3 = self.fc3(output)
        pred4 = self.fc4(output)
        pred5 = self.fc5(output)
        pred1, pred2, pred3, pred4, pred5 = pred1[:, -1, :], pred2[:, -1, :], pred3[:, -1, :], pred4[:, -1, :], pred5[:, -1, :]
        pred1, pred2, pred3, pred4, pred5 = self.softmax(pred1), self.softmax(pred2), self.softmax(pred3), self.softmax(pred4), self.softmax(pred5)
        pred = torch.stack([pred1, pred2, pred3, pred4, pred5], dim=1)
        return pred

In [37]:

model = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [40]:
for epoch in range(config.epochs):
    model.train()
    running_loss = 0
    train_bar = tqdm(train_loader)
    for data in train_bar:
        x_train, y_train = data
        optimizer.zero_grad()
        y_train_pred = model(x_train)
        loss = torch.tensor(0.0)
        for i in range(config.predict_len):
            loss += loss_function(y_train_pred[:, i, :], y_train[:, i])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.desc = 'train epoch[{}/{}] loss: {:.3f}'.format(epoch + 1, config.epochs, loss)
    
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            y_test_pred = model(x_test)
            loss = torch.tensor(0.0)
            for i in range(config.predict_len):
                loss += loss_function(y_train_pred[:, i, :], y_train[:, i])
            test_loss = loss
    
    if test_loss < config.best_loss:
        config.best_loss = test_loss
        torch.save(model.state_dict(), config.save_path)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 123.67it/s]


In [41]:
model.eval()
with torch.no_grad():
    train_bar = tqdm(train_loader)

    predict_rate = [0 for _ in range(config.predict_len)]
    tot = [0 for _ in range(config.predict_len)]
    for data in train_bar:
        x_train, y_train = data
        y_train_pred = model(x_train)
        for i in range(config.predict_len):
            tot[i] += x_train.shape[0]
        for i in range(x_train.shape[0]):
            for j in range(config.predict_len):
                pred = 0
                for k in range(config.output_size):
                    if y_train_pred[i][j][k].item() > y_train_pred[i][j][pred].item():
                        pred = k
                predict_rate[j] += (1 if pred == y_train[i][j] else 0)
    for i in range(config.predict_len):
        print('train set predict rate for step {:d} is: {:f}'.format(i + 1, predict_rate[i] / tot[i]))
    test_bar = tqdm(test_loader)

    predict_rate = [0 for _ in range(config.predict_len)]
    tot = [0 for _ in range(config.predict_len)]
    for data in test_bar:
        x_test, y_test = data
        y_test_pred = model(x_test)
        for i in range(config.predict_len):
            tot[i] += x_test.shape[0]
        for i in range(x_test.shape[0]):
            for j in range(config.predict_len):
                pred = 0
                for k in range(config.output_size):
                    if y_test_pred[i][j][k].item() > y_test_pred[i][j][pred].item():
                        pred = k
                predict_rate[j] += (1 if pred == y_test[i][j] else 0)
    for i in range(config.predict_len):
        print('test set predict rate for step {:d} is: {:f}'.format(i + 1, predict_rate[i] / tot[i]))

100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 67.75it/s]


train set predict rate for step 1 is: 0.372807
train set predict rate for step 2 is: 0.412281
train set predict rate for step 3 is: 0.412281
train set predict rate for step 4 is: 0.390351
train set predict rate for step 5 is: 0.368421


100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 82.14it/s]

test set predict rate for step 1 is: 0.192982
test set predict rate for step 2 is: 0.228070
test set predict rate for step 3 is: 0.228070
test set predict rate for step 4 is: 0.315789
test set predict rate for step 5 is: 0.368421


In [11]:
lst = [np.zeros(1) for _ in range(5)]
np.array(lst).shape

(5, 1)